In [3]:
import pandas as pd
import numpy as np

In [4]:
df_test = pd.read_csv('../data/test.csv')
df_train = pd.read_csv('../data/train.csv')
test_ids = df_test['PassengerId']

In [5]:
def clean(data):
    data = data.drop(['Ticket','Cabin','Name','PassengerId'],axis=1)
    cols = ['SibSp', 'Parch', 'Fare', 'Age']
    for col in cols:
        data[col] = data[col].fillna(data[col].median())

    data.Embarked.fillna('U')
    return data

df_train = clean(df_train)
df_test = clean(df_test)

df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,1,male,39.00,2,0,27.14,S
1,0,3,male,39.00,0,0,13.35,S
2,0,3,male,0.33,1,2,71.29,S
3,0,3,male,19.00,0,0,13.04,S
4,1,3,male,25.00,0,0,7.76,S
...,...,...,...,...,...,...,...,...
99995,1,2,female,62.00,0,0,14.86,C
99996,0,2,male,66.00,0,0,11.15,S
99997,0,3,male,37.00,0,0,9.95,S
99998,0,3,male,51.00,0,1,30.92,S


In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ['Sex', 'Embarked']

for col in cols:
    df_train[col] = le.fit_transform(df_train[col])
    df_test[col] = le.transform(df_test[col])
    print(le.classes_)

df_train.head(5)

['female' 'male']
['C' 'Q' 'S' nan]


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,1,1,39.00,2,0,27.14,2
1,0,3,1,39.00,0,0,13.35,2
2,0,3,1,0.33,1,2,71.29,2
3,0,3,1,19.00,0,0,13.04,2
4,1,3,1,25.00,0,0,7.76,2


In [7]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split

y = df_train['Survived']
X = df_train.drop('Survived', axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train,y_train)

In [9]:
predictions = clf.predict(X_val)

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val,predictions)

0.7684

In [11]:
submission_preds = clf.predict(df_test)

In [14]:
df = pd.DataFrame({'PassengerId': test_ids.values,
                   'Survived': submission_preds,
                  })
df.to_csv('../submission/submission.csv', index=False)